In [1]:
import sqlite3
import pandas as pd
import os
from mido import MidiFile
from midi_player import MIDIPlayer
from midi_player.stylers import basic, cifka_advanced
# import musicbrainzngs

os.chdir('../')
os.chdir('../')

from plagdet.src.defaults import *

import magenta.music as mm
from note_seq.midi_io import note_sequence_to_midi_file
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel
import numpy as np
import os
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

# Necessary until pyfluidsynth is updated (>1.2.5).
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from plagdet.src.embeddings.music_vae.two_bar_encoder import Mel2barEncoderDefault
from plagdet.src.embeddings.music_vae.two_bar_encoder import Mel2barEncoderCustom
from logging import getLogger
import logging

logging.basicConfig(level=logging.ERROR)
logger = getLogger(__name__)

c:\Users\cunn2\miniconda3\envs\magenta\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
c:\Users\cunn2\miniconda3\envs\magenta\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
c:\Users\cunn2\miniconda3\envs\magenta\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find 

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
metadata = pd.read_pickle(SYNTHETIC_PAIR_METADATA_PATH)
tracker = pd.read_csv(SYNTHETIC_PAIR_TRACKER_PATH, index_col='pair_id')

print(metadata.columns)
print(tracker.columns)

Index(['pair_id', 'midi_modifier_config', 'segment_starting_bar',
       'segment_starting_time_secs', 'segment_insertion_starting_bar',
       'segment_insertion_starting_time_secs'],
      dtype='object')
Index(['original_midi_path', 'plagiarised_midi_path'], dtype='object')


In [3]:
pair_id = 10

original_path = tracker.loc[pair_id, 'original_midi_path']
plagiarised_path = tracker.loc[pair_id, 'plagiarised_midi_path']

original_midi = MidiFile(original_path)
plagiarised_midi = MidiFile(plagiarised_path)

metadata_row = metadata.loc[pair_id]

for key, value in metadata_row.items():
    print(f'{key}: {value}')

pair_id: 10
midi_modifier_config: MidiModifierConfig(use_shift_entire_midi_pitch=True, entire_shift_semitone=12, use_shift_selected_notes_pitch=True, selected_notes_pitch_shifts=[((17, 19), -2), ((9, 10), 0)], use_change_note_durations=True, note_scale_factors=[((4, 7), 2.5623164478556983)], use_delete_notes=True, notes_to_delete=[(2, 5)])
segment_starting_bar: 79
segment_starting_time_secs: 158.0
segment_insertion_starting_bar: 10
segment_insertion_starting_time_secs: 20.0


In [4]:
MIDIPlayer(original_path, 500, styler=cifka_advanced, title='Original')

In [5]:
default_encoder = Mel2barEncoderDefault()
custom_encoder = Mel2barEncoderCustom()

custom_vectors = custom_encoder.encode_midis([original_path, plagiarised_path])




INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, CategoricalLstmDecoder, and hparams:
{'max_seq_len': 32, 'z_size': 512, 'free_bits': 0, 'max_beta': 0.5, 'beta_rate': 0.99999, 'batch_size': 4, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [2048, 2048, 2048], 'enc_rnn_size': [2048], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'inverse_sigmoid', 'sampling_rate': 1000, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}


INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, CategoricalLstmDecoder, and hparams:
{'max_seq_len': 32, 'z_size': 512, 'free_bits': 0, 'max_beta': 0.5, 'beta_rate': 0.99999, 'batch_size': 4, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [2048, 2048, 2048], 'enc_rnn_size': [2048], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'inverse_sigmoid', 'sampling_rate': 1000, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}


INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048]



INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048]



INFO:tensorflow:
Decoder Cells:
  units: [2048, 2048, 2048]



...

INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048]



INFO:tensorflow:
Decoder Cells:
  units: [2048, 2048, 2048]



INFO:tensorflow:
Decoder Cells:
  units: [2048, 2048, 2048]



INFO:tensorflow:Restoring parameters from plagdet/models/music_vae/2_bar_melody/cat-mel_2bar_big.ckpt


INFO:tensorflow:Restoring parameters from plagdet/models/music_vae/2_bar_melody/cat-mel_2bar_big.ckpt


In [11]:
import pickle
with open('custom_vectors.pkl', 'wb') as f:
    pickle.dump(custom_vectors, f, protocol=pickle.HIGHEST_PROTOCOL)


In [8]:
dict = custom_vectors['plagdet/data/synthetic_dataset/pairs/original\original_10.mid']

dict['1']


array([-1.85044444e+00,  1.26782447e-01, -1.08765140e-01, -7.25233316e-01,
        1.54024208e+00, -1.25118327e+00, -5.89599729e-01,  1.77868977e-02,
       -1.28185368e+00, -9.37279463e-01,  2.21427751e+00,  5.05004048e-01,
        1.12171859e-01,  1.37708628e+00,  1.19219041e+00, -2.05707431e+00,
       -8.59550416e-01,  6.87113941e-01,  8.43988240e-01,  5.72781980e-01,
        1.45500529e+00,  8.54727477e-02, -1.18875277e+00,  6.31603241e-01,
       -1.77394494e-01, -6.45936430e-01,  7.18122646e-02,  1.36631668e-01,
        8.46209079e-02,  5.54160118e-01, -3.00833285e-01,  1.11987007e+00,
        1.38089371e+00,  1.65632188e-01, -2.82815903e-01, -7.31595978e-03,
        2.55359888e-01, -1.04379380e+00, -8.85328829e-01,  5.05475581e-01,
        9.21505332e-01, -7.50865579e-01,  1.01051569e-01, -2.11292696e+00,
        1.22397602e+00, -2.20143348e-01, -1.24306309e+00,  7.14563951e-02,
        6.24060929e-01,  1.15031183e+00, -2.21902460e-01,  1.18562365e+00,
       -7.83231080e-01, -

In [7]:
import chromadb

client = chromadb.Client()

collection = client.create_collection(MUSIC_VAE_DB)

collection.add(
    ids=[str(pair_id)],
    embeddings=[custom_vectors['plagdet/data/synthetic_dataset/pairs/original\\original_10.mid'][0][0]],
)


ModuleNotFoundError: No module named 'chromadb'